In [2]:
import torch
import torchvision

In [3]:
model = torchvision.models.resnet50()

In [5]:
model.state_dict().keys()

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.we

In [26]:
state["model"]["module.normalizer.new_mean"].size(), state["model"]["module.normalizer.new_std"].size()

(torch.Size([3, 1, 1]), torch.Size([3, 1, 1]))

In [56]:
state = torch.load("/datasets/home/hbenoit/robust_resnet/resnet50_l2_eps0.05.ckpt")
model = RobustModel(torchvision.models.resnet50())
new_state = {}
for k in state["model"]:
    if "attacker" not in k:
        new_state [k.replace("module.","")] = state["model"][k]

    
model.load_state_dict(new_state)

<All keys matched successfully>

In [58]:
model.fc

AttributeError: 'RobustModel' object has no attribute 'fc'

In [38]:
len(model.state_dict().keys())

322

In [43]:
state["model"].keys()

odict_keys(['module.normalizer.new_mean', 'module.normalizer.new_std', 'module.model.conv1.weight', 'module.model.bn1.weight', 'module.model.bn1.bias', 'module.model.bn1.running_mean', 'module.model.bn1.running_var', 'module.model.bn1.num_batches_tracked', 'module.model.layer1.0.conv1.weight', 'module.model.layer1.0.bn1.weight', 'module.model.layer1.0.bn1.bias', 'module.model.layer1.0.bn1.running_mean', 'module.model.layer1.0.bn1.running_var', 'module.model.layer1.0.bn1.num_batches_tracked', 'module.model.layer1.0.conv2.weight', 'module.model.layer1.0.bn2.weight', 'module.model.layer1.0.bn2.bias', 'module.model.layer1.0.bn2.running_mean', 'module.model.layer1.0.bn2.running_var', 'module.model.layer1.0.bn2.num_batches_tracked', 'module.model.layer1.0.conv3.weight', 'module.model.layer1.0.bn3.weight', 'module.model.layer1.0.bn3.bias', 'module.model.layer1.0.bn3.running_mean', 'module.model.layer1.0.bn3.running_var', 'module.model.layer1.0.bn3.num_batches_tracked', 'module.model.layer1.0.

In [50]:
state["model"]["module.model.conv1.weight"][0], state["model"]["module.attacker.model.conv1.weight"][0]

(tensor([[[ 0.0174, -0.0063, -0.0380, -0.0682, -0.0581, -0.0089,  0.0340],
          [-0.0154, -0.0424, -0.0862, -0.1375, -0.1518, -0.1141, -0.0547],
          [ 0.0150,  0.0106, -0.0187, -0.0729, -0.1099, -0.1025, -0.0660],
          [ 0.0463,  0.0699,  0.0720,  0.0330, -0.0268, -0.0492, -0.0463],
          [ 0.0625,  0.1078,  0.1262,  0.1170,  0.0724,  0.0431,  0.0202],
          [ 0.0077,  0.0612,  0.1014,  0.1286,  0.0966,  0.0570,  0.0223],
          [-0.0597, -0.0169,  0.0294,  0.0671,  0.0528,  0.0344,  0.0115]],
 
         [[ 0.0122, -0.0237, -0.0598, -0.0970, -0.0814, -0.0286,  0.0210],
          [-0.0292, -0.0722, -0.1253, -0.1866, -0.1987, -0.1570, -0.0874],
          [ 0.0081, -0.0074, -0.0466, -0.1116, -0.1506, -0.1441, -0.0992],
          [ 0.0528,  0.0757,  0.0748,  0.0267, -0.0414, -0.0701, -0.0672],
          [ 0.0859,  0.1378,  0.1594,  0.1480,  0.0943,  0.0539,  0.0218],
          [ 0.0303,  0.0967,  0.1456,  0.1750,  0.1348,  0.0808,  0.0348],
          [-0.0412,  0

In [29]:
class InputNormalize(torch.nn.Module):
    '''
    A module (custom layer) for normalizing the input to have a fixed 
    mean and standard deviation (user-specified).
    '''
    def __init__(self, new_mean, new_std):
        super(InputNormalize, self).__init__()

        self.register_buffer("new_mean", new_mean)
        self.register_buffer("new_std", new_std)

    def forward(self, x):
        x = torch.clamp(x, 0, 1)
        x_normalized = (x - self.new_mean)/self.new_std
        return x_normalized
    

class RobustModel(torch.nn.Module):

    def __init__(self, model):
        super(RobustModel, self).__init__()
        self.normalizer = InputNormalize(torch.zeros((3,1,1)), torch.zeros((3,1,1)))
        self.model = model

    def forward(self, inp):
        normalized_inp = self.normalizer(inp)
        output = self.model(normalized_inp)
        return output